Profitable App Profiles for the App Store and Google Play Markets
Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

In [1]:
from csv import reader
import csv

# Loading Google play store data
open_file = open("googleplaystore.csv")
read_file = reader(open_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

# Loading Apple Store data
open_file = open("AppleStore.csv")
read_file = reader(open_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]


In [2]:
# Creating a function to print rows repeatedly
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


In [3]:
# Removing Incorrect data

print(android[10472])
print('\n')
print(android_header)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [4]:
print(len(android))
del android[10472]
print(len(android))

10841
10840


In [5]:
#4.1 Printing duplicates

for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [6]:
# 4.2 Count the number of duplicates
unique_apps = []
duplicate_apps = []
for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print("The total number of duplicate apps are: ", len(duplicate_apps))
print('\n')
print(duplicate_apps[:10])

The total number of duplicate apps are:  1181


['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

If you examine the rows we printed for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show the data was collected at different times.

In [7]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.

In [8]:
# 5.1 Creating a dictionary

reviews_max = {}
for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        

In [9]:
# checking the number of entries in dictionary
print("Expected : ", len(android)-1181)
print("Actual : ", len(reviews_max))

Expected :  9659
Actual :  9659


In [10]:
# 5.2 Removing duplicate rows

android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) 

In [11]:
# 5.3 Exploring the new data set

explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


In [12]:
# 6.1 Function for checking english characters

def english(name):
    for char in name:
        if ord(char) > 127:
            return False
        else:
            return True


In [13]:
# 6.2 Checking the output of the function

print(english('Instagram'))
print(english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english('Docs To Go™ Free Office Suite'))
print(english('Instachat 😜'))

True
False
True
True


Removing Non-English Apps

If you explore the data sets enough, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [14]:
# 7.3 Filtering out non english apps from the datasets

android_english_apps = []
ios_english_apps = []

# For Google Play Store

for app in android_clean:
    name = app[0]
    if english(name):
        android_english_apps.append(app)

# For Apple Play Store

for app in ios:
    name = app[1]
    if english(name):
        ios_english_apps.append(app)

        

In [15]:
# 7.4 Exploring the dataset

explore_data(android_english_apps, 0, 3, True)
print("\n")
explore_data(ios_english_apps, 0, 3, True)


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9623
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

In [16]:
# 8.1 Isolating Free Apps

android_new = []
ios_new = []

# Google Play Store
for app in android_english_apps:
    cost = app[7]
    if cost == '0':
        android_new.append(app)

# Apple Play Store
for app in ios_english_apps:
    cost = app[4]
    if cost == '0.0':
        ios_new.append(app)

print(len(android_new))
print(len(ios_new))

8873
3300


In [17]:
# 8.2 Explore the free apps

explore_data(android_new, 0, 3, True)
print('\n')
explore_data(ios_new, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 8873
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

Build a minimal Android version of the app, and add it to Google Play.
If the app has a good response from users, we develop it further.
If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.
Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.

In [18]:
# 10.1 Most common apps by genre
import pandas as pd

# For Google Play

android_new = pd.DataFrame(columns=android_header,data=android_new)
android_new.head()


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
2,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
3,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
4,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up


In [19]:
# For App Store
ios_new = pd.DataFrame(columns = ios_header, data = ios_new)
ios_new.head()

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1
2,529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1
3,420009108,Temple Run,65921024,USD,0.0,1724546,3842,4.5,4.0,1.6.2,9+,Games,40,5,1,1
4,284035177,Pandora - Music & Radio,130242560,USD,0.0,1126879,3594,4.0,4.5,8.4.1,12+,Music,37,4,1,1


In [20]:
# Aggregate based on the column genre for ios
ios_agg = ios_new['prime_genre'].value_counts(normalize= True)*100
ios_agg

Games                57.484848
Entertainment         7.969697
Photo & Video         4.878788
Education             3.575758
Social Networking     3.393939
Utilities             2.696970
Shopping              2.575758
Sports                2.121212
Health & Fitness      2.060606
Music                 2.030303
Productivity          1.757576
Lifestyle             1.696970
News                  1.363636
Travel                1.181818
Finance               1.181818
Weather               0.878788
Food & Drink          0.878788
Book                  0.575758
Business              0.515152
Reference             0.515152
Navigation            0.272727
Medical               0.212121
Catalogs              0.181818
Name: prime_genre, dtype: float64

In [21]:
# Aggregate based on the column Category for android
android_agg = android_new['Category'].value_counts(normalize=True)*100
android_agg

FAMILY                 18.956385
GAME                    9.703595
TOOLS                   8.441339
BUSINESS                4.598219
LIFESTYLE               3.933281
PRODUCTIVITY            3.899470
FINANCE                 3.685338
MEDICAL                 3.527556
SPORTS                  3.392314
PERSONALIZATION         3.324693
COMMUNICATION           3.234532
HEALTH_AND_FITNESS      3.076750
PHOTOGRAPHY             2.952778
NEWS_AND_MAGAZINES      2.817536
SOCIAL                  2.648484
TRAVEL_AND_LOCAL        2.332920
SHOPPING                2.231489
BOOKS_AND_REFERENCE     2.175138
DATING                  1.859574
VIDEO_PLAYERS           1.791953
MAPS_AND_NAVIGATION     1.397498
FOOD_AND_DRINK          1.228446
EDUCATION               1.172095
ENTERTAINMENT           0.946692
LIBRARIES_AND_DEMO      0.935422
AUTO_AND_VEHICLES       0.924152
HOUSE_AND_HOME          0.811450
WEATHER                 0.800180
EVENTS                  0.710019
PARENTING               0.653668
ART_AND_DE

In [22]:
# Aggregate based on the column Genre for android
android_new['Genres'].value_counts(normalize=True)*100

Tools                                    8.430069
Entertainment                            6.063338
Education                                5.375859
Business                                 4.598219
Lifestyle                                3.922011
Productivity                             3.899470
Finance                                  3.685338
Medical                                  3.527556
Sports                                   3.459935
Personalization                          3.324693
Communication                            3.234532
Action                                   3.088020
Health & Fitness                         3.076750
Photography                              2.952778
News & Magazines                         2.817536
Social                                   2.648484
Travel & Local                           2.321650
Shopping                                 2.231489
Books & Reference                        2.175138
Simulation                               2.062437


In [23]:
# Number of user ratings per app Genres for android
pd.options.display.float_format = '{:.2f}'.format
android_new['Installs'] = android_new['Installs'].str[:-1]

android_new['Installs'] = android_new['Installs'].str.replace(',', '')
#android_new.head()
android_new['Installs'] = android_new['Installs'].apply(pd.to_numeric)
android_new.groupby('Genres')['Installs'].sum()

Genres
Action                                  3465886940.00
Action;Action & Adventure                 53000000.00
Adventure                                245417120.00
Adventure;Action & Adventure             106000000.00
Adventure;Education                       10000000.00
Arcade                                  3753681940.00
Arcade;Action & Adventure                 35100001.00
Arcade;Pretend Play                        1000000.00
Art & Design                             112511100.00
Art & Design;Action & Adventure             100000.00
Art & Design;Creativity                    1710000.00
Art & Design;Pretend Play                   500000.00
Auto & Vehicles                           53080061.00
Beauty                                    27197050.00
Board                                    161813110.00
Board;Action & Adventure                   6000000.00
Board;Brain Games                          2850000.00
Books & Reference                       1665936260.00
Books & Reference;Edu

In [24]:
android_new.groupby('Category')['Installs'].sum()

Category
ART_AND_DESIGN          113221100.00
AUTO_AND_VEHICLES        53080061.00
BEAUTY                   27197050.00
BOOKS_AND_REFERENCE    1665936260.00
BUSINESS                696952090.00
COMICS                   43921150.00
COMMUNICATION         11036906201.00
DATING                  140914757.00
EDUCATION               189850000.00
ENTERTAINMENT           988460000.00
EVENTS                   15973160.00
FAMILY                 6193166691.00
FINANCE                 445163132.00
FOOD_AND_DRINK          211728751.00
GAME                  13386814450.00
HEALTH_AND_FITNESS     1143548402.00
HOUSE_AND_HOME           97102461.00
LIBRARIES_AND_DEMO       52995810.00
LIFESTYLE               502544429.00
MAPS_AND_NAVIGATION     497160790.00
MEDICAL                  37732344.00
NEWS_AND_MAGAZINES     2368207260.00
PARENTING                31471010.00
PERSONALIZATION        1529235988.00
PHOTOGRAPHY            4656268915.00
PRODUCTIVITY           5791679314.00
SHOPPING               140032